# Data & Algoritma Understanding

## Data Understanding

### 📊 Nama Dataset
liputan6_data.tar.gz

### 🌍 Languages
- Indonesian

### 🧩 Data Structure
|Nama Kolom|Tipe Data|
|----|--------|
|`id`|`string`|
|`url`|`string`|
|`clean_article`|`string`|
|`clean_summary`|`string`|
|`extractive_summary`|`string`|

### Data Instances
|Nama Kolom|Contoh Data|
|----------|-----------|
|`id`|26408|
|`url`|https://www.liputan6.com/news/read/26408/pbb-siap-membantu-penyelesaian-konflik-ambon|
|`clean_article`|Liputan6.com, Ambon: Partai Bulan Bintang wilayah Maluku bertekad membantu pemerintah menyelesaikan konflik di provinsi tersebut. Syaratnya, penanganan penyelesaian konflik Maluku harus dimulai dari awal kerusuhan, yakni 19 Januari 1999. Demikian hasil Musyawarah Wilayah I PBB Maluku yang dimulai Sabtu pekan silam dan berakhir Senin (31/12) di Ambon. Menurut seorang fungsionaris PBB Ridwan Hasan, persoalan di Maluku bisa selesai asalkan pemerintah dan aparat keamanan serius menangani setiap persoalan di Maluku secara komprehensif dan bijaksana. Itulah sebabnya, PBB wilayah Maluku akan menjadikan penyelesaian konflik sebagai agenda utama partai. PBB Maluku juga akan mendukung penegakan hukum secara terpadu dan tanpa pandang bulu. Siapa saja yang melanggar hukum harus ditindak. Ridwan berharap, Ketua PBB Maluku yang baru, Ali Fauzi, dapat menindak lanjuti agenda politik partai yang telah diamanatkan dan mau mendukung penegakan hukum di Maluku. (ULF/Sahlan Heluth).|
|`clean_summary`|Konflik Ambon telah berlangsung selama tiga tahun. Partai Bulan Bintang wilayah Maluku siap membantu pemerintah menyelesaikan kasus di provinsi tersebut.|
|`extractive_summary`|Liputan6.com, Ambon: Partai Bulan Bintang wilayah Maluku bertekad membantu pemerintah menyelesaikan konflik di provinsi tersebut. Siapa saja yang melanggar hukum harus ditindak.|

### Data Fields
|Nama Kolom|Keterangan|
|----------|----------|
|`id`|Kolom id unique|
|`url`|URL Article|
|`clean_article`|Isi original article|
|`clean_summary`|Ringkasan Abstract|
|`extractive_summary`|Ringkasan Ekstractif|

## Algoritma Understanding

# Model Training & Evaluation

### Load Dataset dan Convert Data


In [1]:
# import pandas as pd
# import glob

In [2]:
# file_list = glob.glob('../data/liputan6_data/canonical/train/*.json')
# df_list = [pd.read_json(f, lines=True) for f in file_list]
# df_train = pd.concat(df_list, ignore_index=True)
# df_train.head()

In [3]:
# df_train.to_csv('../data/train_data.csv', index=False)
# df_train.head()

In [4]:
# file_list = glob.glob('../data/liputan6_data/canonical/dev/*.json')
# df_list = [pd.read_json(f, lines=True) for f in file_list]
# df_dev = pd.concat(df_list, ignore_index=True)
# df_dev.head()

In [5]:
# df_dev.to_csv('../data/dev_data.csv', index=False)
# df_dev.head()

In [6]:
# file_list = glob.glob('../data/liputan6_data/canonical/test/*.json')
# df_list = [pd.read_json(f, lines=True) for f in file_list]
# df_test = pd.concat(df_list, ignore_index=True)
# df_test.head()

In [7]:
# df_test.to_csv('../data/test_data.csv', index=False)
# df_test.head()

In [8]:
# df_train = pd.read_csv('../data/train_data.csv')
# df_train.head()

In [9]:
# df_dev = pd.read_csv('../data/dev_data.csv')
# df_dev.head()

In [10]:
# df_test = pd.read_csv('../data/test_data.csv')
# df_test.head()

## Process


In [11]:
import pandas as pd
import ast
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import BertTokenizer, TFBertModel

### Memuat Data dan Preprocessing

In [ ]:
def parse_list_string(s):
    try:
        return ast.literal_eval(s)
    except (ValueError, SyntaxError):
        return []

# Memuat dataset
print("Memuat data...")
df_all = pd.read_csv('../data/train2_data.csv')

# Membersihkan data
print("Membersihkan data...")
df_all['clean_article'] = df_all['clean_article'].apply(parse_list_string)
df_all['extractive_summary'] = df_all['extractive_summary'].apply(parse_list_string)


print("Membagi data menjadi set Training (60%), Validasi (20%), dan Testing (20%)...")

df_train_val, df_test_articles = train_test_split(
    df_all,
    test_size=0.2,
    random_state=42
)

df_train_articles, df_val_articles = train_test_split(
    df_train_val,
    test_size=0.25,
    random_state=42
)

print(f"Jumlah artikel untuk Training: {len(df_train_articles)}")
print(f"Jumlah artikel untuk Validasi: {len(df_val_articles)}")
print(f"Jumlah artikel untuk Testing: {len(df_test_articles)}")


# Fungsi untuk mengubah format dari per-artikel menjadi per-kalimat
def create_sentence_dataframe(df):
    processed_data = []
    for _, row in df.iterrows():
        article_sentences_tokens = row['clean_article']
        summary_indices = set(row['extractive_summary'])
        sentences_as_strings = [" ".join(sent) for sent in article_sentences_tokens]
        for i, sent_text in enumerate(sentences_as_strings):
            processed_data.append({
                'sentence': sent_text,
                'label': 1 if i in summary_indices else 0,
            })
    return pd.DataFrame(processed_data)

# Buat DataFrame per-kalimat untuk setiap set secara terpisah
print("\nMengubah data menjadi format per-kalimat...")
train_sentences_df = create_sentence_dataframe(df_train_articles)
val_sentences_df = create_sentence_dataframe(df_val_articles)
test_sentences_df = create_sentence_dataframe(df_test_articles)

print(f"Jumlah kalimat di data Training: {len(train_sentences_df)}")
print(f"Jumlah kalimat di data Validasi: {len(val_sentences_df)}")
print(f"Jumlah kalimat di data Testing: {len(test_sentences_df)}")


Memuat data...
Membersihkan data...
Membagi data menjadi set Training (60%), Validasi (20%), dan Testing (20%)...
Jumlah artikel untuk Training: 6582
Jumlah artikel untuk Validasi: 2195
Jumlah artikel untuk Testing: 2195

Mengubah data menjadi format per-kalimat...
Jumlah kalimat di data Training: 76399
Jumlah kalimat di data Validasi: 25819
Jumlah kalimat di data Testing: 26240


### Tokenisasi dengan IndoBERT Tokenizer

In [ ]:
MODEL_NAME = 'indobenchmark/indobert-base-p1'
MAX_LEN = 128

print(f"\nMemuat tokenizer dari {MODEL_NAME}...")
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

def convert_to_bert_input(sentences, tokenizer, max_len):
    return tokenizer.batch_encode_plus(
        sentences.tolist(),
        add_special_tokens=True,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_attention_mask=True
    )

print("\nMelakukan tokenisasi pada semua dataset...")
# DIUBAH: Tokenisasi semua set data secara terpisah
X_train_tokens = convert_to_bert_input(train_sentences_df['sentence'], tokenizer, MAX_LEN)
X_val_tokens = convert_to_bert_input(val_sentences_df['sentence'], tokenizer, MAX_LEN)
X_test_tokens = convert_to_bert_input(test_sentences_df['sentence'], tokenizer, MAX_LEN)

# DIUBAH: Ambil label untuk setiap set secara terpisah
y_train = train_sentences_df['label'].values
y_val = val_sentences_df['label'].values
y_test = test_sentences_df['label'].values


Memuat tokenizer dari indobenchmark/indobert-base-p1...

Melakukan tokenisasi pada semua dataset...


### MEMBANGUN DAN MELATIH MODEL IndoBERT

In [14]:
# pip install tf-keras

In [ ]:
def build_model(bert_model, max_len=128):
    input_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
    sequence_output = bert_model(input_ids, attention_mask=attention_mask)[0]
    cls_token_output = sequence_output[:, 0, :]
    x = tf.keras.layers.Dropout(0.2)(cls_token_output)
    output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output_layer)
    return model

print("\nMemuat pre-trained IndoBERT model...")
bert_model = TFBertModel.from_pretrained(MODEL_NAME)
model = build_model(bert_model, max_len=MAX_LEN)
model.summary()

optimizer = tf.keras.optimizers.AdamW(learning_rate=2e-5)
loss = tf.keras.losses.BinaryCrossentropy()
metrics = [
    tf.keras.metrics.BinaryAccuracy('accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Callbacks
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=1,
    verbose=1,
    mode='min',
    restore_best_weights=True
)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='indobert_summarizer_best_model.h5',
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True,
    verbose=1
)

BATCH_SIZE = 64
EPOCHS = 2 

print("\nMemulai training model...")

# Konversi input menjadi format dictionary yang bisa dibaca model
train_input_dict = {'input_ids': tf.constant(X_train_tokens['input_ids']), 'attention_mask': tf.constant(X_train_tokens['attention_mask'])}
val_input_dict = {'input_ids': tf.constant(X_val_tokens['input_ids']), 'attention_mask': tf.constant(X_val_tokens['attention_mask'])}

history = model.fit(
    train_input_dict,
    y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(val_input_dict, y_val),
    callbacks=[early_stopping, model_checkpoint]
)

print("Training selesai.")


Memuat pre-trained IndoBERT model...



Some layers from the model checkpoint at indobenchmark/indobert-base-p1 were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at indobenchmark/indobert-base-p1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_ids (InputLayer)      [(None, 128)]                0         []                            
                                                                                                  
 attention_mask (InputLayer  [(None, 128)]                0         []                            
 )                                                                                                
                                                                                                  
 tf_bert_model (TFBertModel  TFBaseModelOutputWithPooli   1244413   ['input_ids[0][0]',           
 )                           ngAndCrossAttentions(last_   44         'attention_mask[0][0]']      
                             hidden_state=(None, 128, 7                                       

C:\Users\SUWAAAN\AppData\Roaming\Python\Python312\site-packages\tf_keras\src\engine\training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1194/1194 [==============================] - 13332s 11s/step - loss: 0.4534 - accuracy: 0.8190 - precision: 0.5010 - recall: 0.1410 - val_loss: 0.4252 - val_accuracy: 0.8294 - val_precision: 0.5525 - val_recall: 0.2186
Epoch 2/2
1194/1194 [==============================] - ETA: 0s - loss: 0.4150 - accuracy: 0.8294 - precision: 0.5950 - recall: 0.1825 
Epoch 2: val_loss improved from 0.42518 to 0.42249, saving model to indobert_summarizer_best_model.h5
1194/1194 [==============================] - 14363s 12s/step - loss: 0.4150 - accuracy: 0.8294 - precision: 0.5950 - recall: 0.1825 - val_loss: 0.4225 - val_accuracy: 0.8299 - val_precision: 0.5812 - val_recall: 0.1588
Restoring model weights from the end of the best epoch: 2.
Training selesai.


### Evaluasi Model pada Data Test

In [ ]:
print("\nMemuat model terbaik dari checkpoint untuk evaluasi...")

best_model = tf.keras.models.load_model(
    'indobert_summarizer_best_model.h5', 
    custom_objects={"TFBertModel": TFBertModel}
)

# Siapkan data test
test_input_dict = {'input_ids': tf.constant(X_test_tokens['input_ids']), 'attention_mask': tf.constant(X_test_tokens['attention_mask'])}

print("\nMelakukan evaluasi pada Test Set (data yang belum pernah dilihat)...")
y_pred_proba = best_model.predict(test_input_dict)
y_pred = (y_pred_proba > 0.5).astype(int)

print("\nClassification Report (Test Set):")
print(classification_report(y_test, y_pred, target_names=['Bukan Ringkasan (0)', 'Ringkasan (1)']))


Memuat model terbaik dari checkpoint untuk evaluasi...

Melakukan evaluasi pada Test Set (data yang belum pernah dilihat)...
820/820 [==============================] - 1294s 2s/step

Classification Report (Test Set):
                     precision    recall  f1-score   support

Bukan Ringkasan (0)       0.84      0.98      0.91     21610
      Ringkasan (1)       0.59      0.16      0.25      4630

           accuracy                           0.83     26240
          macro avg       0.72      0.57      0.58     26240
       weighted avg       0.80      0.83      0.79     26240



### INFERENCE (MEMBUAT RINGKASAN)

In [ ]:
def summarize_article(article_sentences, model, tokenizer, max_len, num_sentences=3):
    inputs = tokenizer.batch_encode_plus(
        article_sentences,
        max_length=max_len,
        padding='max_length',
        truncation=True,
        return_tensors='tf',
        add_special_tokens=True
    )
    input_dict = {'input_ids': inputs['input_ids'], 'attention_mask': inputs['attention_mask']}
    probabilities = model.predict(input_dict).flatten()
    top_indices = np.argsort(probabilities)[-num_sentences:]
    top_indices = sorted(top_indices)
    summary = " ".join([article_sentences[i] for i in top_indices])
    return summary


sample_article_data = df_test_articles['clean_article'].iloc[0]
sample_sentences = [" ".join(sent) for sent in sample_article_data]

print(f"\n--- Membuat Ringkasan untuk Artikel Sampel dari Test Set ---")
print("\nArtikel Asli (5 kalimat pertama):")
for sent in sample_sentences[:5]:
    print("- " + sent)


generated_summary = summarize_article(sample_sentences, best_model, tokenizer, max_len=MAX_LEN, num_sentences=3)

print("\nRingkasan yang Dihasilkan Model:")
print(generated_summary)


--- Membuat Ringkasan untuk Artikel Sampel dari Test Set ---

Artikel Asli (5 kalimat pertama):
- Liputan6 . com , Jakarta : Pengamat politik Andi Malarangeng , baru-baru ini , menilai pembaharuan kode etik DPR tidak akan berarti banyak dalam mengubah etika berpolitik para anggota Dewan .
- Sebab , pengaturan masalah etika belum ditetapkan dalam Undang-undang .
- Menurut Andi , muatan kode etik DPR yang akan disampaikan pekan ini masih terlihat samar .
- Selain itu , hubungan antara Pasal Satu dan lainnya tak memiliki konsekuensi yang jelas .
- Akibatnya , kode etik tersebut hanya akan menimbulkan perdebatan .
1/1 [==============================] - 3s 3s/step

Ringkasan yang Dihasilkan Model:
Liputan6 . com , Jakarta : Pengamat politik Andi Malarangeng , baru-baru ini , menilai pembaharuan kode etik DPR tidak akan berarti banyak dalam mengubah etika berpolitik para anggota Dewan . Sebab , para anggota DPR juga terkait dengan kode etik partai . Rencananya , awal Oktober ini , DPR akan 